In [9]:
import pickle
#0:144 result done
with open('output 0 to 144','rb') as f:
     output_0_144 = pickle.load(f)
len(output_0_144)

24

In [10]:
#1973:2117 result done
with open('output 1973 to 2116','rb') as f:
     output_2116_1973 = pickle.load(f)
len(output_2116_1973)

24

In [6]:
#1781:1973 result  done
with open('output 1781 to 1972','rb') as f:
     output_1781_1972 = pickle.load(f)
len(output_1781_1972)

32

In [15]:
#1685:1781 result 
with open('output 1685 to 1780','rb') as f:
     output_1685_1780 = pickle.load(f)
output_1685_1780

[[({'atr_len': 11,
    'macd_fastLen': 12,
    'macd_slowLen': 32,
    'macd_signalSmooth': 8,
    'macd_peakLen': 3,
    'gain_ratio': 1,
    'loss_ratio': 1,
    'peak2_len': 20,
    'peak3_len': 50,
    'ticker': 'SKE'},
   18.187361273268856,
   (36.072269275509676, 0.1450934579439252, 0.75)),
  ({'atr_len': 11,
    'macd_fastLen': 12,
    'macd_slowLen': 32,
    'macd_signalSmooth': 8,
    'macd_peakLen': 3,
    'gain_ratio': 1,
    'loss_ratio': 1,
    'peak2_len': 30,
    'peak3_len': 50,
    'ticker': 'SKE'},
   18.187361273268856,
   (36.072269275509676, 0.1450934579439252, 0.75)),
  ({'atr_len': 12,
    'macd_fastLen': 12,
    'macd_slowLen': 32,
    'macd_signalSmooth': 8,
    'macd_peakLen': 3,
    'gain_ratio': 1,
    'loss_ratio': 1,
    'peak2_len': 20,
    'peak3_len': 50,
    'ticker': 'SKE'},
   18.187361273268856,
   (36.072269275509676, 0.1450934579439252, 0.75))],
 [({'atr_len': 11,
    'macd_fastLen': 11,
    'macd_slowLen': 34,
    'macd_signalSmooth': 7,
    'ma

In [ ]:
# 1589:1685 result done

In [18]:
#1445:1589 result

24

In [ ]:
#1301:1445 result

In [ ]:
#1157:1301 result